# Jupyter Example Interactive Notebook
This Notebook contains different examples for ways to create an interactive online companion.

## Introduction
The aim of this notebook is to show how to export any kind of Jupyter notebook using nbinteract to HTML, interactivity and all. Furthermore it will include several useful examples and tricks to make the most of an interactive notebook.

## Interactive Latex
To deepen a reader's understanding of some equation or formula it is preferable that they are able to interact with it. This could easily be achieved by just simply having them fill in some input boxes and then show an output. However, it looks more professional if they can see the Latex change in front of their eyes. The following text and pieces of code will outline how to achieve this in Jupyter using nbinteract.

### Forcing MathJax updates
When exporting a notebook that contains some Latex that you update dynamically with nbinteract it will not actually render the Latex representation but rather show you the actual code. This is because MathJax, the Javascript library that Jupyter uses to render the math, is never notified of updates your interactive code produces. To make sure MathJax is updates whenever the output of your Python code changes we add the following Javascript to every notebook:

In [2]:
from IPython.display import HTML

HTML('''<script type="text/javascript">
    $('body').on("DOMSubtreeModified", function() {
        MathJax.Hub.Queue(["Typeset", MathJax.Hub, this]);
    });
</script>''')

**Important**

In case you are using the convnb script (explanation in a later section) this cell will not be necessary since the script adds this statically to the generated HTML. When you are not using convnb you need to add a valid jQuery source to the top of the head of the generated HTML. This is because nbinteracts conversion does not provide you with it automatically and the above script will not work if it's not added.

#### Examples
Now, we can use the following code to generate a simple two to the power of $x$ function we can observe the Latex gets rerendered everytime we change $x$.

In [7]:
import ipywidgets as w
import IPython.display as d
import numpy as np

isl = w.IntSlider(
    value=3,
    min=1,
    max=30
)

def disp(x):
    d.display(d.Latex("$2^{" + str(x) + "} = " + str(np.power(2, x)) + "$"))
    
    
output = w.interactive_output(disp, {'x': isl})
d.display(isl, output)

IntSlider(value=3, max=30, min=1)

Output()

This is a rather simple example but the concept is quite powerfull. As a bit bigger example consider example 1.5.1 from the book [Structured Probabilistic Reasoning](http://www.cs.ru.nl/B.Jacobs/PAPERS/ProbabilisticReasoning.pdf) by Bart Jacobs where the following function is introduced:

$$flip(r) := r|1\rangle + (1-r)|0\rangle$$

Here $1$ and $0$ can be viewed as 'Heads' and 'Tails' for a coin flip. Later the book introduces a corresponding bar graph. These can be combined and made interactive in the following way:

In [6]:
from ipywidgets import interactive_output, FloatSlider
from decimal import *
import IPython.display as d
import nbinteract as nbi
import numpy as np

# Round down decimals to 2 positions
getcontext().prec = 2

rSlider = FloatSlider(
    value=0.51,
    min=0,
    max=1,
    step=0.01,
    description='$$r:=$$'
)

options = {
    'title': 'Heads or Tails',
    'xlabel': '',
    'ylabel': 'Chance between 0 and 1 with 1 being 100%',
    'ylim': (0.00, 1.01)
}

def flip(_, r):
    d.display(d.Latex(r'''$$flip(''' + str(r) + ''') := ''' +
                      str(r) + r'''|H\rangle + ''' + str(Decimal(1.0)-Decimal(r)) + r'''|T\rangle$$'''))
    return (float(Decimal(r)), (float(Decimal(1.0)-Decimal(r))))

nbi.bar(['Heads', 'Tails'], flip, r=rSlider, options=options)

As shown above, this is very powerful and can be used to show a great number of examples and equations.

### Drawing TikZ pictures 
It is possible in both a notebook and its HTML extract to draw TikZ pictures using the [TikZJax](https://github.com/kisonecat/tikzjax) library. When only using the HTML extract of a notebook, the convnb script will add the necessary code to the page. If you want to use it inside of a notebook you will need to add the custom.js file located in tools/custom-jupyter-javascript to your ~/.jupyter/custom folder. This javascript contains the necessary machinery that rerenders the TikZ code to a picture.

**Warning:**
TikZJax is limited in what it can and can't do. Be careful to **always** check that your TikZ picture code compiles on its own, without using additional TikZ packages. For additional information go to https://github.com/larsvanarragon/spr-companion/tree/master/tools/custom-jupyter-javascript.

#### Drawing a simple circle
To create a TikZ picture within the notebook simply add a code cell to your notebook that displays and HTML script tag containing the type text/tikz and correct TikZ code. To draw a simple circle your script tag should look as follows:

```
<script type="text/tikz">
  \begin{tikzpicture}
    \draw (0,0) circle (1in);
  \end{tikzpicture}
</script>
```

You can update this HTML by using an interactive output from nbinteract. The machinery in custom.js will detect the new script tag and render it to a picture.

#### Using TikZIt
In case you are using TikZit and have several styles that you'd like your picture to have you will have to add these to the picture locally. This means instead of doing:

```
\tikzstyle{style name}=[some property=x, other=y]
\tikzstyle{another}=[prop=x]
```

You should convert it to:

```
\begin{tikzpicture}[
        style name/.style={some property=x, other=y},
        another/.style={prop=x}]
        ...
\end{tikzpicutre}
```

**Note:** Do not use the property 'tikzit category', TikZJax can't handle this.

**Note:** A conversion of the styles used in the book can be found both in the example below and in tools/tikz-supplements.

#### Example from the book
Using the techniques described above we can draw figure 1.4 from the book as follows:

In [1]:
from IPython.display import HTML

HTML(r'''
<link rel="stylesheet" type="text/css" href="http://tikzjax.com/v1/fonts.css">
<script type="text/tikz">
    \begin{tikzpicture}[
        none/.style={},
        white dot/.style={inner sep=0mm, minimum size=1.5mm, draw=black, shape=circle, text depth=-0.2mm, draw=black, fill=white},
        black dot/.style={inner sep=0mm, minimum size=1.5mm, draw=black, shape=circle, draw=black, fill=black},
        observed/.style={inner sep=0mm, minimum size=5mm, draw=black, shape=circle, text depth=-0.2mm, draw=white, tikzit draw=gray, fill=white},
        latent/.style={inner sep=0mm, minimum size=5mm, draw=black, shape=circle, text depth=-0.2mm, draw=black, fill=white},
        small box/.style={shape=rectangle, text height=1.5ex, text depth=0.25ex, yshift=0.5mm, fill=white, draw=black, minimum height=6mm, yshift=-0.5mm, minimum width=6mm, font={\Large}},
        medium box/.style={shape=rectangle, draw=black, fill=white, small box, minimum width=8mm},
        semilarge box/.style={shape=rectangle, draw=black, fill=white, small box, minimum width=12.5mm},
        large box/.style={shape=rectangle, draw=black, fill=white, small box, minimum width=15mm},
        upground/.style={circuit ee IEC, thick, ground, rotate=90, scale=1.5, inner sep=-2mm, tikzit shape=circle, tikzit fill=blue},
        downground/.style={circuit ee IEC, thick, ground, rotate=-90, scale=1.5, inner sep=-2mm, tikzit shape=circle, tikzit fill=green},
        point/.style={regular polygon, regular polygon sides=3, draw, scale=0.75, inner sep=-0.5pt, minimum width=9mm, fill=white, regular polygon rotate=180},
        copoint/.style={regular polygon, regular polygon sides=3, draw, scale=0.75, inner sep=-0.5pt, minimum width=9mm, fill=white},
        uniform/.style={point, fill=gray, tikzit shape=circle, scale=0.5},
        label/.style={font={\footnotesize}, text height=1.5ex, text depth=0.25ex, tikzit draw=blue, tikzit fill=white},
        left label/.style={label, anchor=east, xshift=2mm, tikzit draw=green, tikzit fill=white},
        right label/.style={label, anchor=west, xshift=-2mm, tikzit draw=purple, tikzit fill=white},
        disintegration/.style={draw=black, fill={gray!50}, tikzit fill=gray, shape=rectangle, minimum width=1.6cm, minimum height=1.2cm, opacity=0.3},
        empty diag/.style={shape=rectangle, draw=darkgray, dashed, minimum width=8mm, minimum height=8mm, yshift=0.5mm},
        diredge/.style={->, >=latex},
        dashed edge/.style={-, dashed}]

        \node [style=black dot] (2) at (0, -2) {};
        \node [style=semilarge box] (4) at (-1.75, 0.5) {sprinkler};
        \node [style=large box] (6) at (-1.5, 3.25) {wet grass};
        \node [style=black dot] (8) at (1.5, 1.5) {};
        \node [style=medium box] (9) at (1.5, 0) {rain};
        \node [style=medium box] (10) at (0, -3.5) {winter};
        \node [style=none] (11) at (1.5, 0.5) {};
        \node [style=none] (12) at (1.5, -0.5) {};
        \node [style=none] (13) at (-1.75, 0) {};
        \node [style=none] (14) at (2.25, 2.75) {};
        \node [style=none] (15) at (-1, 2.75) {};
        \node [style=none] (16) at (-1.75, 2.75) {};
        \node [style=none] (17) at (-1.75, 1) {};
        \node [style=none] (18) at (-1.5, 4.75) {};
        \node [style=none] (20) at (-0.5, -2.25) {$A$};
        \node [style=none] (21) at (-2.25, 1.75) {$B$};
        \node [style=none] (22) at (2, 1.25) {$C$};
        \node [style=none] (23) at (-2, 4.5) {$D$};
        \node [style=none] (24) at (2.75, 4.5) {$E$};
        \node [style=medium box] (25) at (2.25, 3.25) {slippery road};
        \node [style=none] (26) at (2.25, 4.75) {};
        \draw [style=diredge] (10) to (2);
        \draw [style=diredge, in=-90, out=45, looseness=0.75] (2) to (12.center);
        \draw [style=diredge, in=-90, out=135, looseness=0.75] (2) to (13.center);
        \draw [style=diredge] (11.center) to (8);
        \draw [style=diredge, in=-90, out=90, looseness=0.75] (17.center) to (16.center);
        \draw [style=diredge, in=-90, out=135, looseness=0.75] (8) to (15.center);
        \draw [style=diredge, in=-90, out=60, looseness=0.75] (8) to (14.center);
        \draw [style=diredge] (6) to (18.center);
        \draw [style=diredge] (25) to (26.center);
    \end{tikzpicture}
</script>
''')

## Disabling code input blocks
It is not always desirable to show all of the code blocks when the aim is to explain some math problem. Luckily it is quite simple to write some code to hide these from the reader. This can be done as follows:

In [2]:
from IPython.display import HTML

HTML('''<script>
    code_show=false; 
    function code_toggle() {
        if (code_show){
            $('div.input').hide();
        } else {
            $('div.input').show();
        }
        code_show = !code_show
    } 
    $( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()">
    <input type="submit" value="Toggle input code">
</form>''')

In theory this button could be placed anywhere where it is desirable, just apply some CSS.

## The convnb script
When following the examples that are described in this notebook it may be quite tedious to keep having to add jQuery and the Javascript to continuously update the MathJax equations. To automate this we introduce the [convnb](https://github.com/larsvanarragon/nbinteract-tutorial/blob/master/convnb.sh) script. This small script takes care of those little tasks. It can be called in the following way:
```shell
./convnb.sh path/to/jupyter/notebook.ipynb
```
The script generates an HTML file containing all necessary code which can be placed on Github pages.

